# 사용전 가상환경 따로 만들어서 쓸것!


In [ ]:
from pathlib import Path
import sys

# 현재 작업 디렉토리(노트북 파일이 있는 위치)를 기준으로 경로를 설정합니다.
# 예를 들어, 노트북이 my_project/main_app/ 에 있다면...
current_path = Path.cwd()  # my_project/main_app
project_root = current_path.parent  # my_project

# 프로젝트 루트를 sys.path에 추가
sys.path.append(str(project_root))
import json
import pandas as pd
from pathlib import Path
from app.models import (
    GichulSet,
    GichulQna,
    GichulSetType,
    GichulSetInning,
    GichulSetGrade,
    GichulSubject,
)
from app.database import get_db
from sqlmodel import Session, select, SQLModel
from sqlalchemy.engine import Engine
import re

from app.routers.solve import dir_maker, path_getter

In [ ]:
def normalize_gichulset_name(name: str):
    """
    비일관적인 문자열을 정제
    """
    normalized_type = ""
    if "기관사" in name:
        normalized_type = "기관사"
    elif "항해사" in name:
        normalized_type = "항해사"
    elif "소형" in name:
        normalized_type = "소형선박조종사"
    else:
        pass

    split = []
    if normalized_type == "소형선박조종사":
        return (normalized_type, "0")
    else:
        grade = ""
        split = re.split(r"[\.\s?]", name)
        return (normalized_type, split[1].rstrip("급"))


def normalize_gichulqna_qsub(qsub: str):
    split = re.split(r"[\.\s?]", qsub)
    return split[-1]


def insertData(engine: Engine, json_file_path: Path):
    with open(
        json_file_path,
        "r",
        encoding="utf-8",
    ) as f:
        data = json.load(f)
        subject = data["subject"]
        settype, grade = normalize_gichulset_name(subject["name"])
        year = subject["year"]
        inning = subject["inning"]
        with Session(engine) as session:
            set = GichulSet(
                type=GichulSetType(settype),
                grade=GichulSetGrade(grade),
                year=int(year),
                inning=GichulSetInning(str(inning)),
            )
            session.add(set)
            for types in subject["type"]:
                qsubject = normalize_gichulqna_qsub(types["string"])
                for q in types["questions"]:
                    if not q["questionsStr"].strip():
                        continue
                    qna = GichulQna(
                        qnum=int(q["num"]),
                        subject=GichulSubject(qsubject),
                        questionstr=q["questionsStr"],
                        ex1str=q["ex1Str"],
                        ex2str=q["ex2Str"],
                        ex3str=q["ex3Str"],
                        ex4str=q["ex4Str"],
                        answer=q["answer"],
                        gichulset=set,
                    )
                    session.add(qna)
            session.commit()

In [ ]:
# 디렉토리 순회


path = Path("C:/Users/user/Downloads/해기사기출DB(2021-2023)")


def qstrcheck(path: Path):
    with open(
        path,
        "r",
        encoding="utf-8",
    ) as f:
        data = json.load(f)
        df = pd.json_normalize(
            data,
            record_path=["subject", "type", "questions"],
            meta=[
                ["subject", "name"],
                ["subject", "year"],
                ["subject", "inning"],
                ["subject", "type", "string"],
            ],
        )
        display(df.loc[df["questionsStr"].isna()])


engine = run_engine()
# if engine:
for questionfolder in path.glob("*/*"):
    json_file_path = questionfolder / f"{questionfolder.name}.json"
    if not json_file_path.exists():
        print(f"{json_file_path} 존재하지 않음")
        continue

    try:
        qstrcheck(json_file_path)

    except Exception as e:

        print(f"오류: {e} at {json_file_path.name}")

In [21]:
engine = run_engine()


def qstrcheck(path: Path):
    with open(
        path,
        "r",
        encoding="utf-8",
    ) as f:
        data = json.load(f)
        df = pd.json_normalize(
            data,
            record_path=["subject", "type", "questions"],
            meta=[
                ["subject", "name"],
                ["subject", "year"],
                ["subject", "inning"],
                ["subject", "type", "string"],
            ],
        )
        display(df.loc[df["ex2Str"].isna()])


if engine:
    qstrcheck(
        Path(
            "C:/Users/user/Downloads/해기사기출DB(2021-2023)/기관사/E5_2022_02/E5_2022_02.json"
        ),
    )

,num,answer,ex1Str,ex2Str,ex3Str,ex4Str,questionsStr,subject.name,subject.year,subject.inning,subject.type.string


In [3]:
def normalize_gichulset_name(name: str):
    """
    비일관적인 문자열을 정제
    """
    normalized_type = ""
    if "기관사" in name:
        normalized_type = "기관사"
    elif "항해사" in name:
        normalized_type = "항해사"
    elif "소형" in name:
        normalized_type = "소형선박조종사"
    else:
        pass

    split = []
    if normalized_type == "소형선박조종사":
        return (normalized_type, "0")
    else:
        grade = ""
        split = re.split(r"[\.\s?]", name)
        return (normalized_type, split[1].rstrip("급"))


def normalize_gichulqna_qsub(qsub: str):
    split = re.split(r"[\.\s?]", qsub)
    return split[-1]


path = Path("C:/Users/user/Downloads/해기사기출DB(2021-2023)")
for questionfolder in path.glob("*/*"):
    json_file_path = questionfolder / f"{questionfolder.name}.json"
    if not json_file_path.exists():
        print(f"{json_file_path} 존재하지 않음")
        continue
    print(questionfolder.name)

E1_2021_01
E1_2021_02
E1_2021_03
E1_2021_04
E1_2022_01
E1_2022_02
E1_2022_03
E1_2022_04
E1_2023_01
E1_2023_02
E1_2023_03
E1_2023_04
E2_2021_01
E2_2021_02
E2_2021_03
E2_2021_04
E2_2022_01
E2_2022_02
E2_2022_03
E2_2022_04
E2_2023_01
E2_2023_02
E2_2023_03
E2_2023_04
E3_2021_01
E3_2021_02
E3_2021_03
E3_2021_04
E3_2022_01
E3_2022_02
E3_2022_03
E3_2022_04
E3_2023_01
E3_2023_02
E3_2023_03
E3_2023_04
E4_2021_01
E4_2021_02
E4_2021_03
E4_2021_04
E4_2022_01
E4_2022_02
E4_2022_03
E4_2022_04
E4_2023_01
E4_2023_02
E4_2023_03
E4_2023_04
E5_2021_01
E5_2021_02
E5_2021_03
E5_2021_04
E5_2022_01
E5_2022_02
E5_2022_03
E5_2022_04
E5_2023_01
E5_2023_02
E5_2023_03
E5_2023_04
E6_2021_01
E6_2021_02
E6_2021_03
E6_2021_04
E6_2022_01
E6_2022_02
E6_2022_03
E6_2022_04
E6_2023_01
E6_2023_02
E6_2023_03
E6_2023_04
S1_2021_01
S1_2021_02
S1_2021_03
S1_2021_04
S1_2022_01
S1_2022_02
S1_2022_03
S1_2022_04
S1_2023_01
S1_2023_02
S1_2023_03
S1_2023_04
D1_2021_01
D1_2021_02
D1_2021_03
D1_2021_04
D1_2022_01
D1_2022_02
D1_2022_03

In [ ]:
# df.iloc[0]["subject.type.string"].split(" ")[1]
qna = GichulQna(
    qnum=df.iloc[0]["num"],
    subject=df.iloc[0]["subject.type.string"].split(" ")[1],
    questionstr=df.iloc[0]["questionsStr"],
    ex1str=df.iloc[0]["ex1Str"],
    ex2str=df.iloc[0]["ex2Str"],
    ex3str=df.iloc[0]["ex3Str"],
    ex4str=df.iloc[0]["ex4Str"],
    answer=df.iloc[0]["answer"],
)
qna

GichulQna(subject='기관1', qnum='1', questionstr='디젤기관의 연료분사 시 분사지연을 짧게 하는 방법으로 옳은 것은?', ex1str='고압관의 길이를 더 길게 한다.', ex2str='고압관의 두께를 더 두껍게 한다.', ex3str='분사 후 잔류압력을 더 낮게 유지한다.', ex4str='플런저의 면적과 속도를 더 크게 한다.', answer='아', id=None, explanation=None, gichulset_id=None)

In [3]:
engine = run_engine()
with Session(engine) as session:
    set = GichulSet(
        type=GichulSetType(df.iloc[0]["subject.name"].split(" ")[0]),
        grade=GichulSetGrade(df.iloc[0]["subject.name"].split(" ")[1].rstrip("급")),
        year=df.iloc[0]["subject.year"],
        inning=GichulSetInning(df.iloc[0]["subject.inning"]),
    )
    session.add(set)
    qna = GichulQna(
        qnum=df.iloc[0]["num"],
        subject=GichulSubject(df.iloc[0]["subject.type.string"].split(" ")[1]),
        questionstr=df.iloc[0]["questionsStr"],
        ex1str=df.iloc[0]["ex1Str"],
        ex2str=df.iloc[0]["ex2Str"],
        ex3str=df.iloc[0]["ex3Str"],
        ex4str=df.iloc[0]["ex4Str"],
        answer=df.iloc[0]["answer"],
        gichulset=set,
    )
    session.add(qna)
    session.commit()

2025-07-07 12:02:04,787 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-07-07 12:02:04,787 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-07 12:02:04,788 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-07-07 12:02:04,788 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-07 12:02:04,789 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-07-07 12:02:04,789 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-07 12:02:04,790 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-07 12:02:04,792 INFO sqlalchemy.engine.Engine INSERT INTO gichulset (type, grade, year, inning) VALUES (%(type)s, %(grade)s, %(year)s, %(inning)s)
2025-07-07 12:02:04,792 INFO sqlalchemy.engine.Engine [generated in 0.00037s] {'type': '기관사', 'grade': '1', 'year': '2021', 'inning': '1'}
2025-07-07 12:02:04,794 INFO sqlalchemy.engine.Engine INSERT INTO gichulqna (subject, qnum, questionstr, ex1str, ex2str, ex3str, ex4str, answer, explanation, gichulset_id) VALUES (%(subject)s, %(qnum)s, %